In [2]:
# importing the pandas module for
# data frame
!pip install pygam
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from pygam import LinearGAM
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score




# load the data set into train variable.
train = pd.read_csv('/kaggle/input/data-set-test/cars.csv')

# display top 5 values of data set
train.head()



  Using cached pygam-0.9.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached pygam-0.9.1-py3-none-any.whl (522 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


,DateTime,Vehicles
0,1/11/2015 0:00,2
1,1/12/2015 0:00,10
2,1/13/2015 0:00,4
3,1/14/2015 0:00,22
4,1/15/2015 0:00,16


In [3]:
# function to get all data from time stamp

# get date
def get_dom(dt):
	return dt.day

# get week day
def get_weekday(dt):
	return dt.weekday()

# get hour
def get_hour(dt):
	return dt.hour

# get year
def get_year(dt):
	return dt.year

# get month
def get_month(dt):
	return dt.month

# get year day
def get_dayofyear(dt):
	return dt.dayofyear

# get year week
def get_weekofyear(dt):
	return dt.weekofyear


train['DateTime'] = train['DateTime'].map(pd.to_datetime)
train['date'] = train['DateTime'].map(get_dom)
train['weekday'] = train['DateTime'].map(get_weekday)
train['hour'] = train['DateTime'].map(get_hour)
train['month'] = train['DateTime'].map(get_month)
train['year'] = train['DateTime'].map(get_year)
train['dayofyear'] = train['DateTime'].map(get_dayofyear)
train['weekofyear'] = train['DateTime'].map(get_weekofyear)

# display
train.head()


,DateTime,Vehicles,date,weekday,hour,month,year,dayofyear,weekofyear
0,2015-01-11,2,11.0,6.0,0.0,1.0,2015.0,11.0,2.0
1,2015-01-12,10,12.0,0.0,0.0,1.0,2015.0,12.0,3.0
2,2015-01-13,4,13.0,1.0,0.0,1.0,2015.0,13.0,3.0
3,2015-01-14,22,14.0,2.0,0.0,1.0,2015.0,14.0,3.0
4,2015-01-15,16,15.0,3.0,0.0,1.0,2015.0,15.0,3.0


In [4]:
train.to_csv('file1.csv')

In [5]:

# display top 5 values of data set
train.head()

,DateTime,Vehicles,date,weekday,hour,month,year,dayofyear,weekofyear
0,2015-01-11,2,11.0,6.0,0.0,1.0,2015.0,11.0,2.0
1,2015-01-12,10,12.0,0.0,0.0,1.0,2015.0,12.0,3.0
2,2015-01-13,4,13.0,1.0,0.0,1.0,2015.0,13.0,3.0
3,2015-01-14,22,14.0,2.0,0.0,1.0,2015.0,14.0,3.0
4,2015-01-15,16,15.0,3.0,0.0,1.0,2015.0,15.0,3.0


In [6]:
print(train.columns)

Index(['DateTime', ' Vehicles', 'date', 'weekday', 'hour', 'month', 'year',
       'dayofyear', 'weekofyear'],
      dtype='object')


In [7]:
# there is no use of DateTime module
# so remove it
train = train.drop(['DateTime'], axis=1)

# separating class label for training the data
train1 = train.drop([' Vehicles'], axis=1)

# class label is stored in target
target = train[' Vehicles']

print(train1.head())
target.head()


   date  weekday  hour  month    year  dayofyear  weekofyear
0  11.0      6.0   0.0    1.0  2015.0       11.0         2.0
1  12.0      0.0   0.0    1.0  2015.0       12.0         3.0
2  13.0      1.0   0.0    1.0  2015.0       13.0         3.0
3  14.0      2.0   0.0    1.0  2015.0       14.0         3.0
4  15.0      3.0   0.0    1.0  2015.0       15.0         3.0


0     2
1    10
2     4
3    22
4    16
Name:  Vehicles, dtype: int64

In [8]:
target = pd.read_csv('/kaggle/working/file1.csv')

# separating class label for training the data
train1 = train.drop(' Vehicles', axis=1)

# class label is stored in target
target = train[' Vehicles']

train1.head()
#target.head()


,date,weekday,hour,month,year,dayofyear,weekofyear
0,11.0,6.0,0.0,1.0,2015.0,11.0,2.0
1,12.0,0.0,0.0,1.0,2015.0,12.0,3.0
2,13.0,1.0,0.0,1.0,2015.0,13.0,3.0
3,14.0,2.0,0.0,1.0,2015.0,14.0,3.0
4,15.0,3.0,0.0,1.0,2015.0,15.0,3.0


In [36]:

# train1 = pd.read_csv('/kaggle/working/train1.csv')
train1 = train1.fillna(train1.mean())
# target = pd.read_csv('/kaggle/working/target.csv')
#importing Random forest
from sklearn.ensemble import RandomForestRegressor
#defining the RandomForestRegressor

# k = 3
# m1=RandomForestRegressor()
# from catboost import CatBoostRegressor
# m1 = CatBoostRegressor(loss_function='MAE',iterations=10000,verbose=False)

m1 = LinearGAM()

# Training the model
#m1.fit(train1, target)
# m1 = LinearGAM().fit(train1, target)

# m1.fit(train1,target)
#testing
# m1.predict([[11,6,0,1,2015,11,2]])


In [37]:
X_train, X_test, y_train, y_test = train_test_split(train1, target, test_size=0.2, random_state=42)

# Create and train the model
# model = DecisionTreeClassifier(random_state=42)
m1.fit(X_train, y_train)
y_pred = m1.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

Mean Absolute Error: 10.438885254926209
Mean Squared Error: 281.141354639809
Root Mean Squared Error: 16.76727033955763
R-squared: 0.46242876827857626
